In [197]:
import pandas as pd
import yfinance as yf

In [198]:
df = pd.read_csv('Transactions.csv')

In [199]:
# Display the first 5 rows
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))

# Print the column names and their data types
#print(df_transactions.info())

| Datum      | Tijd   | Product                                     | ISIN         | Beurs   | Uitvoeringsplaats   | Aantal   | Koers   | Unnamed: 8   | Lokale waarde   | Unnamed: 10   | Waarde   | Unnamed: 12   | Wisselkoers   | Transactiekosten en/of   | Unnamed: 15   | Totaal   | Unnamed: 17   | Order ID                             |
|:-----------|:-------|:--------------------------------------------|:-------------|:--------|:--------------------|:---------|:--------|:-------------|:----------------|:--------------|:---------|:--------------|:--------------|:-------------------------|:--------------|:---------|:--------------|:-------------------------------------|
| 25-10-2024 | 10:24  | ISHARES MSCI WORLD SRI UCITS ETF            | IE00BYX2JD69 | XET     | XETA                | 46       | 11.46   | EUR          | -527.16         | EUR           | -527.16  | EUR           | nan           | -1                       | EUR           | -528.16  | EUR           | 8b995f92-ab6b-4091-988

In [200]:
df.head()

,Datum,Tijd,Product,ISIN,Beurs,Uitvoeringsplaats,Aantal,Koers,Unnamed: 8,Lokale waarde,Unnamed: 10,Waarde,Unnamed: 12,Wisselkoers,Transactiekosten en/of,Unnamed: 15,Totaal,Unnamed: 17,Order ID
0,25-10-2024,10:24,ISHARES MSCI WORLD SRI UCITS ETF,IE00BYX2JD69,XET,XETA,46.0,11.460,EUR,-527.16,EUR,-527.16,EUR,NaN,-1.0,EUR,-528.16,EUR,8b995f92-ab6b-4091-9886-a49a2123b46e
1,25-10-2024,10:23,ISHARES NASDAQ 100 UCITS ETF EUR HEDGED ACC,IE00BYVQ9F29,XET,XETA,6.0,12.138,EUR,-72.83,EUR,-72.83,EUR,NaN,-3.0,EUR,-75.83,EUR,7556071f-2abd-4b2a-8be3-c664433997d9
2,02-08-2024,14:21,ISHARES NASDAQ 100 UCITS ETF EUR HEDGED ACC,IE00BYVQ9F29,XET,XETA,14.0,11.178,EUR,-156.49,EUR,-156.49,EUR,NaN,-3.0,EUR,-159.49,EUR,4614ecad-4c1a-4626-99a1-9cd1f49a4172
3,02-08-2024,14:17,ISHARES MSCI WORLD SRI UCITS ETF,IE00BYX2JD69,XET,XETA,14.0,10.740,EUR,-150.36,EUR,-150.36,EUR,NaN,-1.0,EUR,-151.36,EUR,4407e0f5-68b6-49cb-95c8-87c257022120
4,17-07-2024,16:36,ISHARES NASDAQ 100 UCITS ETF EUR HEDGED ACC,IE00BYVQ9F29,XET,XETA,13.0,12.022,EUR,-156.29,EUR,-156.29,EUR,NaN,-3.0,EUR,-159.29,EUR,cc0ae71f-426d-46fd-bc07-6c46e240e08e


In [201]:
import yfinance as yf
import pandas as pd
import datetime

df = pd.read_csv('Transactions.csv')

# Convert 'Datum' from string to datetime for filtering and calculations
df['Datum'] = pd.to_datetime(df['Datum'], format='%d-%m-%Y')

# Define a mapping from ISIN to ticker symbols
isin_to_ticker = {
    'IE00BYVQ9F29': 'NQSE.DE',
    'IE00BYX2JD69': '2B7K.DE'
}

# Filter DataFrame to include only the desired ISINs
df_filtered = df[df['ISIN'].isin(isin_to_ticker.keys())].copy()

# Function to get historical price data
def get_historical_price(ticker_symbol, date_str):
    if ticker_symbol:
        try:
            stock = yf.Ticker(ticker_symbol)
            # Use a date range to increase the chance of getting data
            historical_data = stock.history(start=date_str, end=(pd.to_datetime(date_str) + pd.Timedelta(days=1)).strftime('%Y-%m-%d'))
            if not historical_data.empty:
                return historical_data['Close'][0]
            else:
                print(f"${ticker_symbol}: No historical data found for {date_str}.")
        except Exception as e:
            print(f"${ticker_symbol}: Error fetching data - {e}.")
    return None

# Calculate all-time performance
def calculate_all_time_performance(df):
    # Calculate purchase price for each transaction using historical data
    df['Purchase Price'] = df.apply(lambda row: get_historical_price(isin_to_ticker.get(row['ISIN']), row['Datum'].strftime('%Y-%m-%d')), axis=1)

    # Calculate the investment value for each transaction
    df['Investment Value'] = df['Aantal'] * df['Purchase Price']

    # Get current prices (use today's date for current price)
    df['Current Price'] = df.apply(lambda row: get_historical_price(isin_to_ticker.get(row['ISIN']), datetime.datetime.now().strftime('%Y-%m-%d')), axis=1)

    # Calculate current holdings
    df['CurrentHoldings'] = df.groupby('ISIN')['Aantal'].cumsum()

    # Calculate the total investment value for each ISIN
    total_investment = df.groupby('ISIN')['Investment Value'].sum()

    # Get the latest product name for each ISIN
    latest_product_name = df.groupby('ISIN')['Product'].last()

    # Group by ISIN and aggregate other relevant columns
    df_grouped = df.groupby('ISIN').agg(
        CurrentHoldings=('CurrentHoldings', 'last'),
        CurrentValue=('Current Price', 'last')  # Use the last current price
    )

    # Add the TotalInvestment to the grouped DataFrame
    df_grouped['TotalInvestment'] = total_investment

    # Calculate total worth
    df_grouped['Total Worth'] = df_grouped['CurrentHoldings'] * df_grouped['CurrentValue']

    # Calculate performance
    df_grouped['Performance (%)'] = ((df_grouped['Total Worth'] - df_grouped['TotalInvestment']) / df_grouped['TotalInvestment']) * 100

    # Add the product name to the grouped DataFrame
    df_grouped['Product'] = latest_product_name

    # Set the product name as the index
    df_grouped = df_grouped.set_index('Product')

    return df_grouped

# Execute the all-time performance calculation on the filtered DataFrame
performance_df = calculate_all_time_performance(df_filtered.copy())

# Display the all-time performance results
print("All-Time Performance:\n", performance_df)

All-Time Performance:
                                              CurrentHoldings  CurrentValue  \
Product                                                                      
ISHARES NASDAQ 100 UCITS ETF EUR HEDGED ACC             87.0        12.248   
ISHARES MSCI WORLD SRI UCITS ETF                       137.0        11.472   

                                             TotalInvestment  Total Worth  \
Product                                                                     
ISHARES NASDAQ 100 UCITS ETF EUR HEDGED ACC       913.303980  1065.576013   
ISHARES MSCI WORLD SRI UCITS ETF                 1462.697992  1571.664017   

                                             Performance (%)  
Product                                                       
ISHARES NASDAQ 100 UCITS ETF EUR HEDGED ACC        16.672656  
ISHARES MSCI WORLD SRI UCITS ETF                    7.449660  


In [202]:
import yfinance as yf
import pandas as pd
import datetime

df = pd.read_csv('Transactions.csv')

# Convert 'Datum' from string to datetime for filtering and calculations
df['Datum'] = pd.to_datetime(df['Datum'], format='%d-%m-%Y')

# Define a mapping from ISIN to ticker symbols
isin_to_ticker = {
    'IE00BYVQ9F29': 'NQSE.DE',
    'IE00BYX2JD69': '2B7K.DE'
}

# Filter DataFrame to include only the desired ISINs
df_filtered = df[df['ISIN'].isin(isin_to_ticker.keys())].copy()

# Function to get historical price data
def get_historical_price(ticker_symbol, date_str):
    if ticker_symbol:
        try:
            stock = yf.Ticker(ticker_symbol)
            # Use a date range to increase the chance of getting data
            historical_data = stock.history(start=date_str, end=(pd.to_datetime(date_str) + pd.Timedelta(days=1)).strftime('%Y-%m-%d'))
            if not historical_data.empty:
                return historical_data['Close'][0]
            else:
                print(f"${ticker_symbol}: No historical data found for {date_str}.")
        except Exception as e:
            print(f"${ticker_symbol}: Error fetching data - {e}.")
    return None

In [203]:
def calculate_performance(df, start_date, end_date, period_type='all_time'):
    # Filter transactions within the specified date range
    df_period = df[(df['Datum'] >= start_date) & (df['Datum'] <= end_date)].copy()

    # If no transactions in the period, return None
    if df_period.empty:
        return None

    # Calculate purchase price for each transaction using historical data
    df_period['Purchase Price'] = df_period.apply(lambda row: get_historical_price(isin_to_ticker.get(row['ISIN']), row['Datum'].strftime('%Y-%m-%d')), axis=1)

    # Calculate the investment value for each transaction
    df_period['Investment Value'] = df_period['Aantal'] * df_period['Purchase Price']

    # Get current prices (use end_date for current price)
    df_period['Current Price'] = df_period.apply(lambda row: get_historical_price(isin_to_ticker.get(row['ISIN']), end_date.strftime('%Y-%m-%d')), axis=1)

    # Calculate current holdings (corrected)
    df_period['CurrentHoldings'] = df[(df['Datum'] <= end_date)].groupby('ISIN')['Aantal'].cumsum()
    df_period['CurrentHoldings'] = df_period.groupby('ISIN')['CurrentHoldings'].transform(lambda x: x.iloc[-1] if not x.empty else 0)

    # Calculate holdings at the end of the year (for yearly, corrected)
    df_period = df_period.sort_values(by='Datum')
    df_period['HoldingsBoughtInYear'] = df[(df['Datum'] <= end_date)].groupby('ISIN')['Aantal'].cumsum()
    df_period['HoldingsBoughtInYear'] = df_period.groupby('ISIN')['HoldingsBoughtInYear'].transform(lambda x: x.iloc[-1] if not x.empty else 0)

    # Calculate the total investment value for each ISIN
    total_investment = df_period.groupby('ISIN')['Investment Value'].sum()

    # Get the latest product name for each ISIN
    latest_product_name = df_period.groupby('ISIN')['Product'].last()

    # Group by ISIN and aggregate other relevant columns
    df_grouped = df_period.groupby('ISIN').agg(
        CurrentHoldings=('CurrentHoldings', 'last'),
        HoldingsBoughtInYear=('HoldingsBoughtInYear', 'last'),
        CurrentValue=('Current Price', 'last')  # Use the last current price
    )

    # Add the TotalInvestment to the grouped DataFrame
    df_grouped['TotalInvestment'] = total_investment

    # Calculate Total Worth
    if period_type == 'yearly':
        df_grouped['TotalWorthAtEndOfYear'] = df_grouped['HoldingsBoughtInYear'] * df_grouped['CurrentValue']
    else:
        df_grouped['Total Worth'] = df_grouped['CurrentHoldings'] * df_grouped['CurrentValue']

    # Calculate performance
    if period_type == 'yearly':
        df_grouped['Performance (%)'] = ((df_grouped['TotalWorthAtEndOfYear'] - df_grouped['TotalInvestment']) / df_grouped['TotalInvestment']) * 100
    else:
        df_grouped['Performance (%)'] = ((df_grouped['Total Worth'] - df_grouped['TotalInvestment']) / df_grouped['TotalInvestment']) * 100

    # Add the product name to the grouped DataFrame
    df_grouped['Product'] = latest_product_name

    # Set the product name as the index
    df_grouped = df_grouped.set_index('Product')

    # Adjust columns based on period_type
    if period_type == 'yearly':
        df_grouped = df_grouped.drop(columns=['CurrentHoldings'])
        df_grouped = df_grouped.rename(columns={
            'HoldingsBoughtInYear': 'HoldingsBoughtInYear',
            'CurrentValue': 'ValueAtEndOfYear',
        })

    return df_grouped

In [204]:
def calculate_and_store_performance(df_filtered):
    # Calculate all-time performance
    performance_all_time = calculate_performance(df_filtered.copy(), df_filtered['Datum'].min(), datetime.datetime.now(), period_type='all_time')

    # Calculate YTD performance (corrected)
    end_date_ytd = datetime.datetime.now()
    performance_ytd = calculate_performance(df_filtered.copy(), datetime.datetime(end_date_ytd.year, 1, 1), end_date_ytd, period_type='ytd')

    # Calculate yearly performance for 2021, 2022, 2023
    years = [2021, 2022, 2023]
    yearly_performances = {}
    for year in years:
        # Get the last trading day of the year
        last_trading_day = pd.to_datetime(f'{year}-12-31') - pd.tseries.offsets.BDay(n=1)
        performance_yearly = calculate_performance(df_filtered.copy(), datetime.datetime(year, 1, 1), last_trading_day.to_pydatetime(), period_type='yearly')
        if performance_yearly is not None:
            yearly_performances[year] = performance_yearly

    return performance_all_time, performance_ytd, yearly_performances


# --- Main execution ---
performance_all_time, performance_ytd, yearly_performances = calculate_and_store_performance(df_filtered)

# Display the performance results in Pandas DataFrames
print("Total Portfolio Performance:")
print(performance_all_time.to_markdown(numalign="left", stralign="left"))  # Output as Markdown table

print("\nYTD Performance:")
print(performance_ytd.to_markdown(numalign="left", stralign="left"))

print("\nYearly Performance:")
for year, performance_yearly in yearly_performances.items():
    print(f"\n{year}:")
    print(performance_yearly.to_markdown(numalign="left", stralign="left"))

Total Portfolio Performance:
| Product                                     | CurrentHoldings   | HoldingsBoughtInYear   | CurrentValue   | TotalInvestment   | Total Worth   | Performance (%)   |
|:--------------------------------------------|:------------------|:-----------------------|:---------------|:------------------|:--------------|:------------------|
| ISHARES NASDAQ 100 UCITS ETF EUR HEDGED ACC | 87                | 6                      | 12.248         | 913.304           | 1065.58       | 16.6727           |
| ISHARES MSCI WORLD SRI UCITS ETF            | 137               | 46                     | 11.472         | 1462.7            | 1571.66       | 7.44966           |

YTD Performance:
| Product                                     | CurrentHoldings   | HoldingsBoughtInYear   | CurrentValue   | TotalInvestment   | Total Worth   | Performance (%)   |
|:--------------------------------------------|:------------------|:-----------------------|:---------------|:-------------